In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

DATA_DIR = r"C:\Users\PC\Documents\web projects\BN\VK\backend\uploads\brain tumor\Testing"
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
SEED = 123
CLASS_DIRS = ["glioma", "meningioma", "notumor", "pituitary"]
CLASS_NAMES = ["Glioma Tumor", "Meningioma Tumor", "No Tumor", "Pituitary Tumor"]

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=CLASS_DIRS,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=SEED,
 )
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=CLASS_DIRS,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
 )

normalization_layer = layers.Rescaling(1.0 / 255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

Found 1311 files belonging to 4 classes.
Using 1049 files for training.
Found 1311 files belonging to 4 classes.
Using 262 files for validation.


In [9]:
model = keras.Sequential([
    layers.Input(shape=(150, 150, 3)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(4, activation="softmax"),
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,868 (18.42 MB)

 Trainable params: 4,828,868 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
)

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 27s 727ms/step - accuracy: 0.5558 - loss: 1.0652 - val_accuracy: 0.6336 - val_loss: 0.8239
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 508ms/step - accuracy: 0.7264 - loss: 0.6810 - val_accuracy: 0.7519 - val_loss: 0.6228
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 503ms/step - accuracy: 0.8112 - loss: 0.4874 - val_accuracy: 0.7939 - val_loss: 0.5597
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 484ms/step - accuracy: 0.8418 - loss: 0.4035 - val_accuracy: 0.7519 - val_loss: 0.5717
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 478ms/step - accuracy: 0.9009 - loss: 0.2642 - val_accuracy: 0.7901 - val_loss: 0.5749
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 18s 527ms/step - accuracy: 0.9352 - loss: 0.1945 - val_accuracy: 0.7824 - val_loss: 0.6673
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 18s 551ms/step - accuracy: 0.9523 - loss: 0.1352 - val_accuracy: 0.8053 - val_loss: 0.6160
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 517ms/step - accuracy: 0.9390 - loss: 0.1759 - val_accu

Save trained model

In [11]:
model.save(r"C:\Users\PC\Documents\web projects\BN\VK\backend\brain_tumor_model.h5")